In [ ]:
from add_liquidity import *
from constants import *

from remove_liquidity import remove_liquidity_success, remove_liquidity_default_scenario, remove_liquidity
from swap import *
from pool import *

logging.config.fileConfig('logging.ini')

In [ ]:
TICK_LEFT_BOUND = -10000;
TICK_LEFT_MIDDLE_BOUND = 5000;
TICK_RIGHT_MIDDLE_BOUND = 10000;
TICK_RIGHT_BOUND = 15000;

PRICE_LEFT_BOUND = tick_to_price(TICK_LEFT_BOUND)  # 0.3678978
PRICE_LEFT_MIDDLE_BOUND = tick_to_price(TICK_LEFT_MIDDLE_BOUND) # 1.6486800
PRICE_RIGHT_MIDDLE_BOUND = tick_to_price(TICK_RIGHT_MIDDLE_BOUND) # 2.7181459
PRICE_RIGHT_BOUND = tick_to_price(TICK_RIGHT_BOUND) # 4.4813529

In [ ]:
ONE_LP = [(TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10))]

In [ ]:
# add, remove
remove_liquidity_success(Decimal(2), ONE_LP, [], [0], [])
None

In [ ]:
# add, swap, remove
remove_liquidity_success(Decimal(2), ONE_LP, [(Decimal(1), True)], [0], [])
None

In [ ]:
# add, swap, swap, remove (bring price back to original)
remove_liquidity_success(Decimal(2), ONE_LP, [(Decimal(1), True), (Decimal("1.794975288895955090"), False)], [0], [])
None

In [ ]:
# add, remove, add, swap (check whether possible to add and swap after full remove)
remove_liquidity_success(Decimal(2), ONE_LP, [], [0], [(Decimal(1), True)], ONE_LP)
None

In [ ]:
TWO_LP_OVERLAPPING_INSIDE = [
    (TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10)),
    (TICK_LEFT_MIDDLE_BOUND, TICK_RIGHT_MIDDLE_BOUND, Decimal(10), Decimal(10))
]

# add, add, remove, remove (multiple consequitive remove)
remove_liquidity_success(Decimal(2), TWO_LP_OVERLAPPING_INSIDE, [], [0, 0], [])
None

In [ ]:
# test correct updating of active tick after remove liquidity
# ADD (outer) with price in the middle
# ADD (inner)
# REMOVE (inner)
# SELL beyond left middle bound (active tick is now in inconsistent state if not updated)
# ADD with left bound between price and left middle bound
# BUY beyond at least left bound of newly added liquidity (or beyond previous active tick)

price = Decimal(2)
price_sqrt = price.sqrt()
added_positions = []
for (left_bound, right_bound, x_amount, y_amount) in TWO_LP_OVERLAPPING_INSIDE: # ADD both, REMOVE second = only ADD first
    L, _, _ = add_liquidity(price, left_bound, right_bound, x_amount, y_amount)
    added_positions.append((L, left_bound, right_bound))
remove_L, remove_left_tick, remove_right_tick = added_positions.pop(1)
remove_liquidity(remove_L, price_sqrt ** 2, remove_left_tick, remove_right_tick)
price_sqrt, output, remainder, _ = calculate_swap(added_positions, price_sqrt, Decimal(2), True) # SELL
for (left_bound, right_bound, x_amount, y_amount) in TWO_LP_OVERLAPPING_INSIDE[1:]:  # ADD second position again
    L, _, _ = add_liquidity(price_sqrt**2, left_bound, right_bound, x_amount, y_amount)
    added_positions.append((L, left_bound, right_bound))
price_sqrt, output, remainder, _ = calculate_swap(added_positions, price_sqrt, Decimal(10), False) # BUY

In [ ]:
# add, remove with MAX_SUPPLY
ONE_LP_MAX_SUPPLY = [(TICK_LEFT_BOUND, TICK_RIGHT_BOUND, MAX_SUPPLY, MAX_SUPPLY)]
remove_liquidity_success(Decimal(2), ONE_LP_MAX_SUPPLY, [], [0], [])
None

In [ ]:
# All following examples will follow this pattern
# add, add, remove, swap, remove

In [ ]:
TWO_LP_IDENTICAL = [
    (TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10)),
    (TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10))
]

In [ ]:
# two lp identical - price lesser / equal left bound
remove_liquidity_default_scenario(Decimal("0.3"), TWO_LP_IDENTICAL)
None

In [ ]:
# two lp identical - price between bounds
remove_liquidity_default_scenario(Decimal(2), TWO_LP_IDENTICAL)
None

In [ ]:
# two lp identical - price greater / equal right bound
remove_liquidity_default_scenario(PRICE_RIGHT_BOUND, TWO_LP_IDENTICAL)
None

In [ ]:
liquidity_python = decimal.Decimal("6.6208731164376104264363674850772245712682214635267991765229191370")

liquidity = decimal.Decimal("6.6208731164376104264363674850772245712682214635267991765229191363")
pls = decimal.Decimal("0.6065458221578347578405131291196676381010272902296962274902328579")
prs = decimal.Decimal("2.116920635892453415903744380311046607359547161998842524737018411")
price_sqrt = decimal.Decimal("1.6040505561343578095683588001901790342718967223408928622677446809")
print(price_sqrt)
# price_sqrt = decimal.Decimal("2.5729781866349425743001881278971065388886321877696093977340582393137247712563228065710574092988611781115933699634644914167965065461750041566089409539595063247270786263386546628189280123361538712801381975195114994971519569742102906939670318316836959616405891358994682219865054581928850836216074251582713693857305134374938481854850816968646807716653321133069823075585527590500446846989811615133096535793949388104268168379777839214309047484092190153360628643245459817612210785179521853437816960075533277").sqrt()
# print(price_sqrt)
x_amount = liquidity / price_sqrt - liquidity / prs
y_amount = liquidity * (price_sqrt - pls)

print(x_amount)
print(y_amount)

In [ ]:
TWO_LP_SEPARATE = [
    (TICK_LEFT_BOUND, TICK_LEFT_MIDDLE_BOUND, Decimal(10), Decimal(10)),
    (TICK_RIGHT_MIDDLE_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10))
]

In [ ]:
# two lp separate - price lesser / equal left bound
remove_liquidity_default_scenario(Decimal("0.3"), TWO_LP_SEPARATE)
None

In [ ]:
# two lp separate - price between left bounds
remove_liquidity_default_scenario(Decimal("1.0001"), TWO_LP_SEPARATE)
None

In [ ]:
# two lp separate - price equal left middle bound
remove_liquidity_default_scenario(PRICE_LEFT_MIDDLE_BOUND, TWO_LP_SEPARATE)

In [ ]:
# two lp separate - price between middle bounds
remove_liquidity_default_scenario(Decimal(2), TWO_LP_SEPARATE)

In [ ]:
# two lp separate - price equal right middle bound
remove_liquidity_default_scenario(PRICE_RIGHT_MIDDLE_BOUND, TWO_LP_SEPARATE)

In [ ]:
# two lp separate - price between right bounds
remove_liquidity_default_scenario(Decimal(3), TWO_LP_SEPARATE)

In [ ]:
# two lp separate - price greater/equal right bound
remove_liquidity_default_scenario(PRICE_RIGHT_BOUND, TWO_LP_SEPARATE)

In [ ]:
TWO_LP_DIRECT_NEIGHBORS = [
    (TICK_LEFT_BOUND, TICK_LEFT_MIDDLE_BOUND, Decimal(10), Decimal(10)),
    (TICK_LEFT_MIDDLE_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10))
]

In [ ]:
# two lp direct neighbors - price lesser / equal left bound
remove_liquidity_default_scenario(Decimal("0.3"), TWO_LP_DIRECT_NEIGHBORS)

In [ ]:
# two lp direct neighbors - price between left bounds
remove_liquidity_default_scenario(Decimal("1.0001"), TWO_LP_DIRECT_NEIGHBORS)

In [ ]:
print(f'{decimal.Decimal("0.00000000000000000000000000009239248394839483948394853").sqrt():f}')

In [ ]:
# two lp direct neighbors - price equal middle bound
remove_liquidity_default_scenario(PRICE_LEFT_MIDDLE_BOUND, TWO_LP_DIRECT_NEIGHBORS)

In [ ]:
# two lp direct neighbors - price between right bounds
remove_liquidity_default_scenario(Decimal(2), TWO_LP_DIRECT_NEIGHBORS)

In [ ]:
# two lp direct neighbors - price equal / greater right bound
remove_liquidity_default_scenario(PRICE_RIGHT_BOUND, TWO_LP_DIRECT_NEIGHBORS)

In [ ]:
TWO_LP_OVERLAPPING_EXACT_LEFT = [
    (TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10)),
    (TICK_LEFT_BOUND, TICK_LEFT_MIDDLE_BOUND, Decimal(10), Decimal(10))
]

In [ ]:
# two lp overlapping exact left - price lesser / equal left bound
remove_liquidity_default_scenario(PRICE_LEFT_BOUND, TWO_LP_OVERLAPPING_EXACT_LEFT, amount=1)

In [ ]:
# two lp overlapping exact left - price between left bounds
remove_liquidity_default_scenario(Decimal("1.0001"), TWO_LP_OVERLAPPING_EXACT_LEFT)

In [ ]:
# two lp overlapping exact left - price equal middle bound
remove_liquidity_default_scenario(PRICE_LEFT_MIDDLE_BOUND, TWO_LP_OVERLAPPING_EXACT_LEFT)

In [ ]:
# two lp overlapping exact left - price between right bounds
remove_liquidity_default_scenario(Decimal(2), TWO_LP_OVERLAPPING_EXACT_LEFT)

In [ ]:
# two lp overlapping exact left - price equal / greater right bound
remove_liquidity_default_scenario(PRICE_RIGHT_BOUND, TWO_LP_OVERLAPPING_EXACT_LEFT, amount=3)

In [ ]:
TWO_LP_OVERLAPPING_INSIDE = [
    (TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10)),
    (TICK_LEFT_MIDDLE_BOUND, TICK_RIGHT_MIDDLE_BOUND, Decimal(10), Decimal(10))
]

In [ ]:
# two lp overlapping inside - price lesser / equal left bound
remove_liquidity_default_scenario(PRICE_LEFT_BOUND, TWO_LP_OVERLAPPING_INSIDE, amount=1)

In [ ]:
# two lp overlapping inside - price between left bounds
remove_liquidity_default_scenario(Decimal("1.0001"), TWO_LP_OVERLAPPING_INSIDE, amount=1)

In [ ]:
# two lp overlapping inside - price equal left middle bound
remove_liquidity_default_scenario(PRICE_LEFT_MIDDLE_BOUND, TWO_LP_OVERLAPPING_INSIDE)

In [ ]:
# two lp overlapping inside - price between middle bounds
remove_liquidity_default_scenario(Decimal(2), TWO_LP_OVERLAPPING_INSIDE, amount="0.5")

In [ ]:
# two lp overlapping inside - price equal right middle bound
remove_liquidity_default_scenario(PRICE_RIGHT_MIDDLE_BOUND, TWO_LP_OVERLAPPING_INSIDE, amount="1")

In [ ]:
# two lp overlapping inside - price between right bounds
remove_liquidity_default_scenario(Decimal(3), TWO_LP_OVERLAPPING_INSIDE)

In [ ]:
# two lp overlapping inside - price equal / greater right bound
remove_liquidity_default_scenario(PRICE_RIGHT_BOUND, TWO_LP_OVERLAPPING_INSIDE)

In [ ]:
TWO_LP_OVERLAPPING_EXACT_RIGHT = [
    (TICK_LEFT_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10)),
    (TICK_RIGHT_MIDDLE_BOUND, TICK_RIGHT_BOUND, Decimal(10), Decimal(10))
]

In [ ]:
# two lp overlapping exact right bound - price lesser / equal left bound
remove_liquidity_default_scenario(PRICE_LEFT_BOUND, TWO_LP_OVERLAPPING_EXACT_RIGHT, amount=9)

In [ ]:
# two lp overlapping exact right bound - price between left bounds
remove_liquidity_default_scenario(Decimal("1.0001"), TWO_LP_OVERLAPPING_EXACT_RIGHT, amount=13)

In [ ]:
# two lp overlapping exact right bound - price equal middle bound
remove_liquidity_default_scenario(PRICE_RIGHT_MIDDLE_BOUND, TWO_LP_OVERLAPPING_EXACT_RIGHT)

In [ ]:
# two lp overlapping exact right bound - price between right bounds
remove_liquidity_default_scenario(Decimal(3), TWO_LP_OVERLAPPING_EXACT_RIGHT, amount="0.2")

In [ ]:
# two lp overlapping exact right bound - price greater / equal right bounds
remove_liquidity_default_scenario(PRICE_RIGHT_BOUND, TWO_LP_OVERLAPPING_EXACT_RIGHT, amount="0.5")